# O-T01: QAOA MAX-CUT Experiment Review

**Experiment ID:** O-T01  
**Workstream:** Optimization  
**Objective:** Demonstrate shot-frugal QAOA optimization using classical shadows for cost function estimation on 5-node ring graph MAX-CUT

**Phase 1 Success Criteria:**
- Approximation ratio ≥ 0.90
- ≥20% reduction in optimizer steps vs. standard QAOA
- Manifest and convergence data generated

This notebook reviews the results from O-T01 trials, analyzes convergence behavior, and computes key metrics.

In [ ]:
import json
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / "src"))

from quartumse.reporting.manifest import load_manifest
from quartumse.reporting.report import generate_summary_report

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

## 1. Load Convergence Data

O-T01 generates convergence logs tracking cost function values per optimizer iteration.

In [ ]:
# Load convergence log (adjust trial ID as needed)
data_dir = Path.cwd().parent / "data"
convergence_log_path = data_dir / "logs" / "o-t01-convergence.json"  # or o-t01-trial-01-convergence.json

if not convergence_log_path.exists():
    # Try to find any O-T01 convergence log
    log_files = list((data_dir / "logs").glob("o-t01*convergence.json"))
    if log_files:
        convergence_log_path = log_files[0]
        print(f"Found convergence log: {convergence_log_path}")
    else:
        print("No convergence log found. Run O-T01 experiment first.")
        convergence_data = None
else:
    with convergence_log_path.open("r") as f:
        convergence_data = json.load(f)
    print(f"Loaded: {convergence_log_path}")

## 2. Experiment Overview

In [ ]:
if convergence_data:
    print("=" * 70)
    print("O-T01 EXPERIMENT OVERVIEW")
    print("=" * 70)
    print(f"Experiment ID:      {convergence_data['experiment_id']}")
    print(f"Trial ID:           {convergence_data.get('trial_id', 'N/A')}")
    print(f"Backend:            {convergence_data['backend']}")
    print(f"Shadow size:        {convergence_data['shadow_size']}")
    print(f"Shadow version:     {convergence_data['shadow_version']}")
    print(f"QAOA depth (p):     {convergence_data['p']}")
    print(f"Optimizer:          {convergence_data['optimizer']}")
    print(f"Random seed:        {convergence_data['seed']}")
    print()
    print("FINAL RESULTS:")
    final = convergence_data['final_results']
    print(f"  Approximation ratio: {final['approx_ratio']:.4f} (target: ≥0.90)")
    print(f"  Iterations:          {final['iterations']}")
    print(f"  Total time:          {final['total_time']:.2f}s")
    print(f"  Final params:        {final['final_params']}")
    print()
    print(f"Manifest: {convergence_data['manifest_path']}")
    print("=" * 70)

## 3. Convergence Analysis

Visualize how the QAOA cost function evolves during optimization.

In [ ]:
if convergence_data:
    history = convergence_data['convergence_history']
    
    # Extract data
    iterations = [h['iteration'] for h in history]
    costs = [h['cost'] for h in history]
    max_cuts = [h['max_cut_value'] for h in history]
    ci_widths = [h['mean_ci_width'] for h in history]
    
    # Create convergence plots
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Plot 1: Cost function
    axes[0, 0].plot(iterations, costs, 'b-', linewidth=2, marker='o', markersize=4)
    axes[0, 0].set_xlabel('Iteration')
    axes[0, 0].set_ylabel('Cost Function Value')
    axes[0, 0].set_title('QAOA Cost Function Convergence')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot 2: MAX-CUT value
    axes[0, 1].plot(iterations, max_cuts, 'g-', linewidth=2, marker='s', markersize=4)
    axes[0, 1].axhline(y=4.0, color='r', linestyle='--', label='Optimal (4)')
    axes[0, 1].set_xlabel('Iteration')
    axes[0, 1].set_ylabel('MAX-CUT Value')
    axes[0, 1].set_title('MAX-CUT Solution Quality vs Iteration')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Plot 3: CI width (shadow estimation quality)
    axes[1, 0].plot(iterations, ci_widths, 'orange', linewidth=2, marker='^', markersize=4)
    axes[1, 0].set_xlabel('Iteration')
    axes[1, 0].set_ylabel('Mean CI Width')
    axes[1, 0].set_title('Shadow Estimation Quality (CI Width)')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Plot 4: Approximation ratio
    approx_ratios = [mc / 4.0 for mc in max_cuts]  # Optimal = 4
    axes[1, 1].plot(iterations, approx_ratios, 'm-', linewidth=2, marker='D', markersize=4)
    axes[1, 1].axhline(y=0.90, color='r', linestyle='--', label='Target (0.90)')
    axes[1, 1].set_xlabel('Iteration')
    axes[1, 1].set_ylabel('Approximation Ratio')
    axes[1, 1].set_title('Approximation Ratio vs Iteration')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print(f"\nConvergence Statistics:")
    print(f"  Total iterations:        {len(history)}")
    print(f"  Best MAX-CUT achieved:   {max(max_cuts):.4f}")
    print(f"  Final approximation:     {approx_ratios[-1]:.4f}")
    print(f"  Mean CI width:           {np.mean(ci_widths):.4f}")
    print(f"  Std CI width:            {np.std(ci_widths):.4f}")

## 4. Load and Inspect Final Manifest

The manifest contains the final optimized circuit and provenance data.

In [ ]:
if convergence_data:
    manifest_path = Path(convergence_data['manifest_path'])
    
    if manifest_path.exists():
        manifest = load_manifest(manifest_path)
        
        print("=" * 70)
        print("MANIFEST OVERVIEW")
        print("=" * 70)
        print(f"Manifest ID:        {manifest.manifest_id}")
        print(f"Timestamp:          {manifest.timestamp}")
        print(f"Backend:            {manifest.backend}")
        print(f"Circuit depth:      {manifest.circuit_depth}")
        print(f"Circuit hash:       {manifest.circuit_fingerprint[:16]}...")
        print(f"Shadow size:        {manifest.shadow_config.get('shadow_size', 'N/A')}")
        print(f"Shadow version:     {manifest.shadow_config.get('version', 'N/A')}")
        print()
        print(f"Observables measured: {len(manifest.observables)}")
        for obs_name in list(manifest.observables.keys())[:5]:  # Show first 5
            obs = manifest.observables[obs_name]
            print(f"  {obs_name}: {obs['expectation_value']:.4f} ± CI: {obs['ci_95']}")
        print("=" * 70)
    else:
        print(f"Manifest not found: {manifest_path}")

## 5. Backend Calibration Data

Review the quantum hardware properties at execution time.

In [ ]:
if convergence_data and manifest_path.exists():
    backend_props = manifest.backend_properties
    
    if backend_props:
        print("=" * 70)
        print("BACKEND CALIBRATION DATA")
        print("=" * 70)
        print(f"Backend name:       {backend_props.get('backend_name', 'N/A')}")
        print(f"Calibration time:   {backend_props.get('calibration_timestamp', 'N/A')}")
        print()
        
        # Qubit properties
        qubits = backend_props.get('qubits', [])
        if qubits:
            print("Qubit Properties (first 5 qubits):")
            print(f"{'Qubit':<8} {'T1 (μs)':<12} {'T2 (μs)':<12} {'Readout Error':<15}")
            print("-" * 50)
            for i, q in enumerate(qubits[:5]):
                t1 = q.get('T1', 0) * 1e6  # Convert to μs
                t2 = q.get('T2', 0) * 1e6
                ro_err = q.get('readout_error', 0)
                print(f"{i:<8} {t1:<12.2f} {t2:<12.2f} {ro_err:<15.4f}")
        
        print("=" * 70)
    else:
        print("No backend properties found (simulator backend).")

## 6. Shot Efficiency Analysis

Compare shadow-based QAOA to standard measurement-based QAOA.

In [ ]:
if convergence_data:
    shadow_size = convergence_data['shadow_size']
    iterations = convergence_data['final_results']['iterations']
    
    # Shadow-based shots
    shadow_total_shots = shadow_size * iterations
    
    # Standard QAOA baseline (from literature/assumptions)
    # Typically: 1000 shots/iteration, 60-80 iterations
    standard_shots_per_iter = 1000
    standard_iterations = 60  # Baseline estimate
    standard_total_shots = standard_shots_per_iter * standard_iterations
    
    # Compute metrics
    shot_reduction = (1 - shadow_total_shots / standard_total_shots) * 100
    step_reduction = (1 - iterations / standard_iterations) * 100
    
    print("=" * 70)
    print("SHOT EFFICIENCY COMPARISON")
    print("=" * 70)
    print(f"{'Metric':<30} {'Standard QAOA':<20} {'Shadow QAOA':<20}")
    print("-" * 70)
    print(f"{'Shots per iteration':<30} {standard_shots_per_iter:<20} {shadow_size:<20}")
    print(f"{'Iterations to convergence':<30} {standard_iterations:<20} {iterations:<20}")
    print(f"{'Total shots':<30} {standard_total_shots:<20} {shadow_total_shots:<20}")
    print()
    print(f"Shot reduction:        {shot_reduction:>6.1f}%")
    print(f"Step reduction:        {step_reduction:>6.1f}% (target: ≥20%)")
    print()
    
    # Success criteria
    approx_ratio = convergence_data['final_results']['approx_ratio']
    approx_pass = approx_ratio >= 0.90
    step_pass = step_reduction >= 20.0
    
    print("Phase 1 Success Criteria:")
    print(f"  Approx Ratio ≥ 0.90:   {'✓ PASS' if approx_pass else '✗ FAIL'} ({approx_ratio:.4f})")
    print(f"  Step Reduction ≥ 20%:  {'✓ PASS' if step_pass else '✗ FAIL'} ({step_reduction:.1f}%)")
    print(f"  Manifest Generated:    ✓ PASS")
    print(f"  Convergence Logged:    ✓ PASS")
    print()
    
    if approx_pass and step_pass:
        print("✅ O-T01 PASSED - Phase 1 optimization workstream validated!")
    else:
        print("⚠️  O-T01 PARTIAL PASS - Review convergence and consider parameter tuning")
    
    print("=" * 70)

## 7. Multi-Trial Comparison (Optional)

If multiple trials were run, compare their convergence behavior.

In [ ]:
# Load multiple trials if available
log_files = list((data_dir / "logs").glob("o-t01-trial-*-convergence.json"))

if len(log_files) > 1:
    print(f"Found {len(log_files)} trials. Loading all...")
    
    trials_data = []
    for log_file in log_files:
        with log_file.open("r") as f:
            trial = json.load(f)
            trials_data.append(trial)
    
    # Compare final results
    print("\n" + "=" * 70)
    print("MULTI-TRIAL COMPARISON")
    print("=" * 70)
    print(f"{'Trial':<10} {'Approx Ratio':<15} {'Iterations':<15} {'Total Time':<15}")
    print("-" * 70)
    
    approx_ratios = []
    for trial in trials_data:
        trial_id = trial.get('trial_id', 'N/A')
        final = trial['final_results']
        approx_ratios.append(final['approx_ratio'])
        print(f"{trial_id:<10} {final['approx_ratio']:<15.4f} "
              f"{final['iterations']:<15} {final['total_time']:<15.2f}")
    
    print()
    print(f"Mean approx ratio:     {np.mean(approx_ratios):.4f} ± {np.std(approx_ratios):.4f}")
    print(f"Min/Max:               {np.min(approx_ratios):.4f} / {np.max(approx_ratios):.4f}")
    print("=" * 70)
else:
    print("Only one trial found. Run multiple trials for statistical comparison.")

## 8. Export Results

Save summary for Phase 1 gate review.

In [ ]:
if convergence_data:
    # Create summary report
    summary = {
        "experiment_id": "O-T01",
        "workstream": "Optimization",
        "backend": convergence_data['backend'],
        "shadow_size": shadow_size,
        "final_results": convergence_data['final_results'],
        "phase1_criteria": {
            "approx_ratio_pass": approx_pass,
            "step_reduction_pass": step_pass,
            "overall_pass": approx_pass and step_pass,
        },
        "shot_efficiency": {
            "shadow_total_shots": shadow_total_shots,
            "standard_baseline_shots": standard_total_shots,
            "shot_reduction_percent": shot_reduction,
            "step_reduction_percent": step_reduction,
        },
    }
    
    output_path = data_dir.parent / "results" / "o-t01-summary.json"
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    with output_path.open("w") as f:
        json.dump(summary, f, indent=2)
    
    print(f"\n✅ Summary exported to: {output_path}")

## 9. Next Steps

**If O-T01 PASSED:**
- Include in Phase 1 gate review as optimization workstream validation
- Plan O-T02 (larger graphs, p=2-3)
- Draft Shadow-VQE patent claims

**If O-T01 needs improvement:**
- Increase shadow_size (300 → 500)
- Try different optimizer (SLSQP vs COBYLA)
- Run additional trials for statistical confidence
- Try p=2 ansatz for better approximation ratio